<!-- This section initializes data for the timetable and room allocations. -->

<!-- Timetable Data Initialization -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
    <h3>Timetable Data Initialization</h3>
    <p>This section initializes data for the timetable. It defines the following:</p>
    <ul>
        <li><strong>Time slots</strong></li>
        <li><strong>Days</strong></li>
        <li><strong>Sections</strong></li>
        <li><strong>Course names</strong></li>
        <li><strong>Professors</strong></li>
    </ul>
    <p>Professors are assigned to courses while ensuring that each professor teaches a maximum of three courses. Unique combinations of courses and sections are generated, with each section having a random strength and being designated as either Theory or Lab. The information, including course name, section, section strength, and assigned professor, is then saved to an Excel file named "<strong>schedule.xlsx</strong>".</p>
</div>

<!-- Room Data Initialization -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">
    <h3>Room Data Initialization</h3>
    <p>Additionally, room data is initialized, defining rooms with the following:</p>
    <ul>
        <li><strong>Room number</strong></li>
        <li><strong>Floor number</strong></li>
        <li><strong>Maximum capacity</strong></li>
    </ul>
    <p>Fifteen rooms are created, each with a random floor association ranging from 1 to 3 and a capacity of either 60 or 120. This information, including room number, floor number, and maximum capacity, is saved to an Excel file named "<strong>room_capacities_with_floor.xlsx</strong>".</p>
</div>


In [ ]:
import pandas as pd
import random

Slots = ["08:30", "10:00", "11:30", "01:00", "02:30", "04:00"]
Days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
Sections = [chr(65 + i) for i in range(5)]  # 5 sections from A to E
Course_Names = [
    "Introduction to Computer Science",
    "Data Structures and Algorithms",
    "Database Management",
    "Software Engineering",
    "Computer Networks",
    "Artificial Intelligence",
    "Web Development",
    "Mobile Application Development",
    "Operating Systems",
    "Cybersecurity"
]
Professors = [
    "Prof. Ahmed", "Prof. Ali", "Prof. Sana", "Prof. Hina", "Prof. Imran",
    "Prof. Farah", "Prof. Omar", "Prof. Zara", "Prof. Asad", "Prof. Ayesha",
    "Prof. Sadia", "Prof. Naveed", "Prof. Amina", "Prof. Yasir", "Prof. Rabia",
    "Prof. Usman", "Prof. Ayesha", "Prof. Bilal", "Prof. Saima", "Prof. Fahad"
]
UniqueClasses = []

# Create combinations of courses and sections to form unique classes
for section in Sections:
    section_courses = random.sample(Course_Names, 5)  # Each section has 5 courses
    for course_name in section_courses:
        section_strength = random.randint(50, 120)  # Random section strength
        theory_lab = random.choices(["Theory", "Lab"], weights=[7, 3])[0]  # Random Theory/Lab designation
        UniqueClasses.append({
            "Course_Name": course_name,
            "Section": section,
            "Theory_Lab": theory_lab,
            "Section_Strength": section_strength
        })

# Create DataFrame from UniqueClasses
df = pd.DataFrame(UniqueClasses)

# Shuffle the list of professors
random.shuffle(Professors)

# Dictionary to keep track of assigned courses for each professor
professor_courses_count = {professor: 0 for professor in Professors}

# Assign professors to each row while ensuring maximum of 3 courses per professor
for index, row in df.iterrows():
    assigned_professor = None
    while not assigned_professor:
        selected_professor = random.choice(Professors)
        if professor_courses_count[selected_professor] < 3:
            assigned_professor = selected_professor
            professor_courses_count[selected_professor] += 1
    df.at[index, 'Professor'] = assigned_professor

# stores the information Course Name, Section, Section Strength, Professor Assigned.
file_name = "schedule.xlsx"
df.to_excel(file_name, index=False)
print("DataFrame successfully saved to", file_name)


#Working for Rooms and their related information.
# Define the Rooms list
Rooms = [("Room " + str(i), random.randint(1, 3), random.choice([60, 120])) for i in range(1, 11)]  # 15 rooms with random floor association (1-3) and random capacity (60 or 120)
# Create DataFrame for room capacities
room_df = pd.DataFrame(Rooms, columns=["Room_Number", "Floor_Number", "Max_Capacity"])
room_file_name = "room_capacities_with_floor.xlsx"
room_df.to_excel(room_file_name, index=False)
print("DataFrame successfully saved to", room_file_name)
# print(room_df)






<!-- Load Course Schedule Data -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
    <h3>Load Course Schedule Data</h3>
    <p>This section loads the course schedule data from the Excel file "<strong>schedule.xlsx</strong>".</p>
</div>

<!-- Load Room Capacities Data -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
    <h3>Load Room Capacities Data</h3>
    <p>This section loads the room capacities data from the Excel file "<strong>room_capacities_with_floor.xlsx</strong>".</p>
</div>

<!-- Define Chromosome Creation Function -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">
    <h3>Define Chromosome Creation Function</h3>
    <p>This function creates a chromosome in binary encoding based on the provided entry from the schedule dataframe. It extracts information such as course name, theory/lab designation, section, section strength, professor, lecture days, timeslots, and room details. This information is encoded into binary format and stored in the chromosome dictionary.</p>
</div>

<!-- Define Integer to Binary Conversion Function -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">
    <h3>Define Integer to Binary Conversion Function</h3>
    <p>This function converts an integer to a binary string with a specified number of bits, ensuring that leading zeros are added if necessary.</p>
</div>

<!-- Create Instance Function -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">
    <h3>Create Instance Function</h3>
    <p>This function creates instances of chromosomes by encoding all entries in the schedule dataframe using the <strong>create_chromosome_binary</strong> function. The encoded chromosomes are stored in a list and returned for further processing.</p>
</div>


In [19]:
# Load course schedule data
schedule_df = pd.read_excel("schedule.xlsx")
# Load room capacities data
room_df = pd.read_excel("room_capacities_with_floor.xlsx")


# Define a function to create a chromosome in binary encoding
def create_chromosome_binary(entry, previous_day=None):
    chromosome = {}

    # Extract information from the entry
    course = entry['Course_Name']
    theory_lab = entry['Theory_Lab']
    section = entry['Section']
    section_strength = entry['Section_Strength']  # Extract section strength
    professor = entry['Professor']

    # Encode course name as integer index
    course_index = Course_Names.index(course)
    course_binary = int_to_binary(course_index, 4)

    # Select random lecture days and timeslots for the first lecture
    first_lecture_day = random.choice(Days)
    first_lecture_timeslot = random.choice(Slots)

    # If the course is a lab, ensure it's not scheduled on the same or adjacent day
    if theory_lab == "Lab":
        days_without_adjacent = [day for day in Days if day not in [first_lecture_day, previous_day]]
        second_lecture_day = random.choice(days_without_adjacent)
        # Find the index of the first lecture timeslot
        first_timeslot_index = Slots.index(first_lecture_timeslot)
        # Ensure that the second lecture timeslot is the consecutive next
        second_lecture_timeslot_index = (first_timeslot_index + 1) % len(Slots)
        second_lecture_timeslot = Slots[second_lecture_timeslot_index]
    else:
        # For theory courses, ensure they are not scheduled on the same or adjacent day
        days_without_adjacent = [day for day in Days if day not in [first_lecture_day, previous_day]]
        second_lecture_day = random.choice(days_without_adjacent)
        # Select random lecture timeslots for the second lecture
        second_lecture_timeslot = random.choice(Slots)

    # Select random room and room size
    room_entry = random.choice(Rooms)
    first_lecture_room = room_entry[0]  # Room name
    first_lecture_floor = room_entry[1]  # Floor number
    first_lecture_room_size = room_entry[2]  # Capacity

    # Select another random room and room size
    room_entry = random.choice(Rooms)
    second_lecture_room = room_entry[0]  # Room name
    second_lecture_floor = room_entry[1]  # Floor number
    second_lecture_room_size = room_entry[2]  # Capacity

    # Encode section, professor
    section_binary = int_to_binary(Sections.index(section), 4)
    professor_binary = int_to_binary(Professors.index(professor), 5)

    # Encode theory/lab
    theory_lab_binary = "0" if theory_lab == "Theory" else "1"

    # Encode section strength
    section_strength_binary = int_to_binary(section_strength, 7)  # Assuming a maximum section strength of 127

    # Encode lecture days and timeslots
    first_lecture_day_binary = int_to_binary(Days.index(first_lecture_day), 3)
    first_lecture_timeslot_binary = int_to_binary(Slots.index(first_lecture_timeslot), 4)
    second_lecture_day_binary = int_to_binary(Days.index(second_lecture_day), 3)
    second_lecture_timeslot_binary = int_to_binary(Slots.index(second_lecture_timeslot), 4)

    # Encode room information
    first_lecture_room_binary = int_to_binary(Rooms.index((first_lecture_room, first_lecture_floor, first_lecture_room_size)), 4)
    first_lecture_floor_binary = int_to_binary(first_lecture_floor, 2)
    first_lecture_room_size_binary = "0" if first_lecture_room_size == 60 else "1"

    second_lecture_room_binary = int_to_binary(Rooms.index((second_lecture_room, second_lecture_floor, second_lecture_room_size)), 4)
    second_lecture_floor_binary = int_to_binary(second_lecture_floor, 2)
    second_lecture_room_size_binary = "0" if second_lecture_room_size == 60 else "1"

    # Assign information to the chromosome
    chromosome['Course'] = course_binary
    chromosome['Theory/Lab'] = theory_lab_binary
    chromosome['Section'] = section_binary
    chromosome['Section_Strength'] = section_strength_binary  # Add section strength to the chromosome
    chromosome['Professor'] = professor_binary
    chromosome['First_lecture_day'] = first_lecture_day_binary
    chromosome['First_lecture_timeslot'] = first_lecture_timeslot_binary
    chromosome['First_lecture_room'] = first_lecture_room_binary
    chromosome['First_lecture_floor'] = first_lecture_floor_binary
    chromosome['First_lecture_room_size'] = first_lecture_room_size_binary
    chromosome['Second_lecture_day'] = second_lecture_day_binary
    chromosome['Second_lecture_timeslot'] = second_lecture_timeslot_binary
    chromosome['Second_lecture_room'] = second_lecture_room_binary
    chromosome['Second_lecture_floor'] = second_lecture_floor_binary
    chromosome['Second_lecture_room_size'] = second_lecture_room_size_binary

    return chromosome

# Convert integer to binary string with leading zeros
def int_to_binary(num, num_bits):
    binary = bin(num)[2:]
    return binary.zfill(num_bits)



def create_instance():
  # Create a list to store all chromosomes
  chromosomes = []
  # Encode all entries in the schedule dataframe
  for index, entry in schedule_df.iterrows():
      chromosome = create_chromosome_binary(entry)
      chromosomes.append(chromosome)
  return chromosomes

<!-- Import Required Libraries -->

<!-- Define Genetic Algorithm Parameters -->


<!-- Define Evaluate Population Function -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
    <h3>Define Evaluate Population Function</h3>
    <p>This function evaluates the fitness of each individual in the population by applying specific criteria to assess the quality of generated timetables.</p>
</div>

<!-- Define Calculate Fitness Function -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
    <h3>Define Calculate Fitness Function</h3>
    <p>This function calculates the fitness score of a given timetable based on constraints such as professor and room availability, section schedules, and other requirements.</p>
</div>

<!-- Define Select Fittest Individuals Function -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
    <h3>Define Select Fittest Individuals Function</h3>
    <p>This function selects the most suitable individuals from the population based on their fitness scores to proceed to the next generation, ensuring the evolution of better timetables over iterations.</p>
</div>

<!-- Define Generate Initial Population Function -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
    <h3>Define Generate Initial Population Function</h3>
    <p>This function generates the initial population of timetables, creating a diverse set of solutions to start the genetic algorithm.</p>
</div>

<!-- Define Selection Function -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
    <h3>Define Selection Function</h3>
    <p>This function selects individuals from the population based on their fitness scores, utilizing a tournament selection process to prioritize individuals with higher fitness.</p>
</div>

<!-- Define Crossover Function -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
    <h3>Define Crossover Function</h3>
    <p>This function performs crossover (recombination) between selected parents to produce offspring, allowing the exchange of genetic information between solutions to create potentially better solutions.</p>
</div>

<!-- Define Mutation Function -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
    <h3>Define Mutation Function</h3>
    <p>This function introduces random changes (mutations) in the chromosomes of individuals, adding diversity to the population and potentially leading to the discovery of better solutions.</p>
</div>

<!-- Define Genetic Algorithm Function -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
    <h3>Define Genetic Algorithm Function</h3>
    <p>This function implements the main genetic algorithm, orchestrating the initialization, selection, crossover, mutation, and evolution of the population over multiple generations to optimize the timetable generation process.</p>
</div>

<!-- Decode Chromosome Function -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
    <h3>Decode Chromosome Function</h3>
    <p>This function decodes the binary-encoded chromosome into readable timetable information, allowing for the interpretation of solution representations in human-readable format.</p>
</div>

<!-- Save Timetable to File Function -->
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
    <h3>Save Timetable to File Function</h3>
    <p>This function saves the final timetable to CSV files for each day, providing a structured representation of the optimized timetable solution for practical use.</p>
</div>


In [ ]:
from random import randint
import pandas as pd
# Genetic Algorithm

# Define genetic algorithm parameters
population_size = 100
num_generations = 1000
mutation_rate = 0.2
r_cross = 0.8


def evaluate_population(population):
    fitness_scores = []
    for timetable in population:
        fitness = calculate_fitness(timetable)
        fitness_scores.append(fitness)
    return fitness_scores


def calculate_fitness(timetable):
    conflicts = 0

    # Dictionary to track professors' schedules
    professor_schedule = {}

    # Dictionary to track sections' schedules
    section_schedule = {}

    # Dictionary to track rooms' schedules
    room_schedule = {}

    # Dictionary to track lectures per course per week
    course_lectures = {}

    for chromosome in timetable:  # Loop through each chromosome in the timetable
        for lecture in ['First', 'Second']:
            lecture_day_key = f'{lecture}_lecture_day'
            lecture_timeslot_key = f'{lecture}_lecture_timeslot'
            lecture_room_key = f'{lecture}_lecture_room'
            lecture_section_key = 'Section'

            # Get details of the lecture from the chromosome
            lecture_day = Days[int(chromosome[lecture_day_key], 2)]
            lecture_timeslot = int(chromosome[lecture_timeslot_key], 2)  # Convert to integer
            lecture_room = Rooms[int(chromosome[lecture_room_key], 2)]
            lecture_section = Sections[int(chromosome[lecture_section_key], 2)]
            lecture_professor = Professors[int(chromosome['Professor'], 2)]
            lecture_course = Course_Names[int(chromosome['Course'], 2)]

            # Check professor schedule conflict
            if lecture_professor in professor_schedule:
                if (lecture_day, lecture_timeslot) in professor_schedule[lecture_professor]:
                    conflicts += 1
                else:
                    professor_schedule[lecture_professor].append((lecture_day, lecture_timeslot))
            else:
                professor_schedule[lecture_professor] = [(lecture_day, lecture_timeslot)]

            # Check section schedule conflict
            section_timeslot = (lecture_day, lecture_timeslot)
            if lecture_section in section_schedule:
                if section_timeslot in section_schedule[lecture_section]:
                    conflicts += 1
                else:
                    section_schedule[lecture_section].append(section_timeslot)
            else:
                section_schedule[lecture_section] = [section_timeslot]

            # Check room schedule conflict
            room_timeslot = (lecture_day, lecture_timeslot)
            if lecture_room in room_schedule:
                if room_timeslot in room_schedule[lecture_room]:
                    conflicts += 1
                else:
                    room_schedule[lecture_room].append(room_timeslot)
            else:
                room_schedule[lecture_room] = [room_timeslot]

            # Soft constraint: All theory classes should be in the morning session
            if 'Theory' in lecture and lecture_timeslot >= 3:  # Morning session ends at index 3 (2:30)
                conflicts += 1
            elif 'Lab' in lecture and lecture_timeslot < 3:  # Lab sessions should be in the afternoon session
                conflicts += 1

            # Hard constraint: Each course should have two lectures per week not on the same or adjacent days
            if 'Theory' in lecture:
                if lecture_course in course_lectures:
                    if lecture_day in course_lectures[lecture_course] or Days.index(lecture_day) - 1 in course_lectures[lecture_course]:
                        conflicts += 1
                    else:
                        course_lectures[lecture_course].append(Days.index(lecture_day))
                else:
                    course_lectures[lecture_course] = [Days.index(lecture_day)]

    # Calculate fitness as negative of the sum of conflicts
    fitness = -conflicts
    return fitness



def select_fittest_individuals(population, fitness_scores):
    # Find the indices of the individuals with the maximum fitness scores
    fittest_indices = sorted(range(len(fitness_scores)), key=lambda i: fitness_scores[i], reverse=True)[:2]

    # Return the corresponding individuals from the population
    return [population[index] for index in fittest_indices]



def generate_initial_population(population_size):
    population = []
    for _ in range(population_size):
        population.append(create_instance())
    return population

def evaluate_population(population):
    fitness_scores = []
    for timetable in population:
        fitness = calculate_fitness(timetable)
        fitness_scores.append(fitness)
    return fitness_scores

def selection(population, scores, k=5):
    # Initialize the index of the selected parent
    selection_ix = randint(0, len(population) - 1)


    # Perform tournament selection
    for _ in range(k - 1):
        # Randomly select another individual from the population
        ix = randint(0, len(population) - 1)

        # Compare the fitness scores of the two individuals
        if scores[ix] < scores[selection_ix]:
            # If the new individual has a better fitness score, update the selection index
            selection_ix = ix

    # Return the selected parent from the population
    return population[selection_ix]

def crossover(p1, p2):
    # Children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()

    # Check for recombination
    if random.random() < r_cross:
        # Select crossover point that is not on the end of the string
        pt = randint(1, len(p1) - 2)

        # Perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]

    return c1, c2

# Define function to introduce noise in the chromosome
def mutation(chromosome):
    # print("yes")
    # Define noise probabilities for each chromosome key
    noise_probabilities = {
        'first_lecture_day': 0.2,
        'first_lecture_timeslot': 0.2,
        'second_lecture_day': 0.2,
        'second_lecture_timeslot': 0.2,
        'second_lecture_room': 0.2
    }

    # Introduce noise for first lecture day
    if random.random() < noise_probabilities['first_lecture_day']:
        chromosome['First_lecture_day'] = int_to_binary(random.randint(0, len(Days) - 1), 3)

    # Introduce noise for first lecture timeslot
    if random.random() < noise_probabilities['first_lecture_timeslot']:
        chromosome['First_lecture_timeslot'] = int_to_binary(random.randint(0, len(Slots) - 1), 4)

    # Introduce noise for second lecture day
    if random.random() < noise_probabilities['second_lecture_day']:
        chromosome['Second_lecture_day'] = int_to_binary(random.randint(0, len(Days) - 1), 3)

    # Introduce noise for second lecture timeslot
    if random.random() < noise_probabilities['second_lecture_timeslot']:
        chromosome['Second_lecture_timeslot'] = int_to_binary(random.randint(0, len(Slots) - 1), 4)

    # Introduce noise for second lecture room
    if random.random() < noise_probabilities['second_lecture_room']:
        chromosome['Second_lecture_room'] = int_to_binary(random.randint(0, len(Rooms) - 1), 5)

    return chromosome


def genetic_algorithm(num_generations):

    avg_fitness_per_generation = []
    best_timetable = None  # Variable to store the best timetable instance
    best_fitness = float('-inf')  # Variable to store the best fitness found

    # Generate initial population
    population = generate_initial_population(population_size)

    # Main loop for the genetic algorithm
    for generation in range(num_generations):
        # Evaluate fitness of each chromosome in the population
        fitness_scores = evaluate_population(population)

        # Calculate average fitness of the current population
        avg_fitness = sum(fitness_scores) / len(fitness_scores)
        avg_fitness_per_generation.append(avg_fitness)
        print(f"Average fitness at generation {generation + 1}: {avg_fitness}")

        selected_parents = []
        for _ in range(2):  # Select two parents
            parent = selection(population, fitness_scores)
            selected_parents.append(parent)

        # print("Selected Parents Fitness:")
        # for parent in selected_parents:
        #     fitness = calculate_fitness(parent)
        #     print(fitness)

        # Perform crossover on selected parents
        children = crossover(selected_parents[0], selected_parents[1])

        # # Print the children after crossover
        # print("Children after crossover Fitness:")
        # for child in children:
        #     fitness = calculate_fitness(child)
        #     print(fitness)

        # Introduce noise in a randomly selected chromosome of the children
        mutated_children = []
        for child in children:
            mutated_child = child  # Initialize mutated child as a copy of the original child
            if random.random() < mutation_rate:
                # Select a random index (chromosome) from the child timetable
                index = random.randint(0, len(child) - 1)
                # Apply mutation only to the selected chromosome
                mutated_child[index] = mutation(mutated_child[index])
            mutated_children.append(mutated_child)

        # Print the children after introducing noise
        # print("Children after introducing noise Fitness:")
        # for noisy_child in  mutated_children:
        #     fitness = calculate_fitness(noisy_child)
        #     print(fitness)

        # Replace parent if children have a better fitness
        for i in range(len(selected_parents)):
            if fitness_scores[i] < calculate_fitness(mutated_children[i]):
                population.remove(selected_parents[i])
                population.append(mutated_children[i])



     # Find the instance with the highest fitness (smallest negative value)
    max_fitness_index = fitness_scores.index(max(fitness_scores))
    best_timetable_from_population = population[max_fitness_index]

    return avg_fitness_per_generation, best_timetable_from_population


avg_fitness_per_generation, best_timetable_from_population = genetic_algorithm(population_size)

def decode_chromosome(chromosome):
    decoded_chromosome = {
        'Course_Name': Course_Names[int(str(chromosome['Course']), 2)],
        'Theory_Lab': 'Theory' if str(chromosome['Theory/Lab']) == '0' else 'Lab',
        'Section': Sections[int(str(chromosome['Section']), 2)],
        'Professor': Professors[int(str(chromosome['Professor']), 2)],
        'First_Lecture_Day': Days[int(str(chromosome['First_lecture_day']), 2)],
        'First_Lecture_Timeslot': Slots[int(str(chromosome['First_lecture_timeslot']), 2)],
        'First_Lecture_Room': Rooms[int(str(chromosome['First_lecture_room']), 2)][0],
        'First_Lecture_Floor': Rooms[int(str(chromosome['First_lecture_room']), 2)][1],
        'First_Lecture_Room_Size': 60 if chromosome['First_lecture_room_size'] == '0' else 120,
        'Second_Lecture_Day': Days[int(str(chromosome['Second_lecture_day']), 2)],
        'Second_Lecture_Timeslot': Slots[int(str(chromosome['Second_lecture_timeslot']), 2)],
        'Second_Lecture_Room': Rooms[int(str(chromosome['Second_lecture_room']), 2)][0],
        'Second_Lecture_Floor': Rooms[int(str(chromosome['Second_lecture_room']), 2)][1],
        'Second_Lecture_Room_Size': 60 if chromosome['Second_lecture_room_size'] == '0' else 120
    }
    return decoded_chromosome


def save_timetable_to_file(best_timetable):
    # Initialize an empty dictionary to store DataFrames for each day
    timetable_per_day = {day: pd.DataFrame(columns=["Room"] + Slots) for day in Days}

    # Fill in the timetable DataFrames with the available lectures for each day
    for chromosome in best_timetable:
        decoded_chromosome = decode_chromosome(chromosome)
        # Fill in the timetable DataFrame for the first lecture
        day1 = decoded_chromosome['First_Lecture_Day']
        room1 = decoded_chromosome['First_Lecture_Room']
        timeslot1 = decoded_chromosome['First_Lecture_Timeslot']
        course_info1 = f"{decoded_chromosome['Course_Name']}, Prof: {decoded_chromosome['Professor']}, Section: {decoded_chromosome['Section']}, {decoded_chromosome['Theory_Lab']}"
        timetable_per_day[day1].loc[room1, timeslot1] = course_info1
        # Fill in the timetable DataFrame for the second lecture
        day2 = decoded_chromosome['Second_Lecture_Day']
        room2 = decoded_chromosome['Second_Lecture_Room']
        timeslot2 = decoded_chromosome['Second_Lecture_Timeslot']
        course_info2 = f"{decoded_chromosome['Course_Name']}, Prof: {decoded_chromosome['Professor']}, Section: {decoded_chromosome['Section']}, {decoded_chromosome['Theory_Lab']}"
        timetable_per_day[day2].loc[room2, timeslot2] = course_info2
        # For Lab, occupy two consecutive cells if not the last slot
        if decoded_chromosome['Theory_Lab'] == 'Lab' and timeslot2 != Slots[-1]:
            timetable_per_day[day2].loc[room2, Slots[Slots.index(timeslot2) + 1]] = course_info2

    # Ensure all rooms are included in the timetable for each day
    for day, timetable_df in timetable_per_day.items():
        all_rooms = [f"Room {i}" for i in range(1, 11)]  # List of all rooms
        missing_rooms = set(all_rooms) - set(timetable_df.index)  # Find rooms not used for lectures
        for room in missing_rooms:
            timetable_df.loc[room] = ""  # Fill in empty string for missing rooms
        # Sort the rows by room number
        timetable_df = timetable_df.reindex(all_rooms)

        # Rename the columns to include start and end times
        new_columns = []
        for slot in Slots:
            start_time = slot
            end_time = (pd.to_datetime(start_time) + pd.Timedelta(minutes=80)).strftime("%H:%M")
            new_columns.append(f"{start_time}-{end_time}")
        timetable_df.columns = ["Room"] + new_columns

        # Save the timetable DataFrame to a CSV file
        filename = f"{day}_timetable.csv"
        timetable_df.to_csv(filename)
        print(f"Timetable for {day} saved to {filename}")

# Assuming best_timetable_from_population is the variable holding the best timetable obtained from the genetic algorithm
save_timetable_to_file(best_timetable_from_population)








In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define a function to run the genetic algorithm with different population sizes
def run_experiment(population_size):
    avg_fitness_per_generation, _ = genetic_algorithm(population_size=population_size)
    return avg_fitness_per_generation

# Define population sizes for the experiments
population_sizes = [10, 100, 1000]

# Initialize a dictionary to store average fitness per generation for each population size
avg_fitness_per_population = {}

# Run experiments for each population size
for population_size in population_sizes:
    avg_fitness_per_population[population_size] = run_experiment(population_size)

# Plotting
generations = np.arange(1, num_generations + 1)

plt.figure(figsize=(10, 6))

# Plot average fitness per generation for each population size
for population_size, avg_fitness in avg_fitness_per_population.items():
    plt.plot(generations, avg_fitness, label=f"Population Size: {population_size}")

plt.title("Average Fitness per Generation for Different Population Sizes")
plt.xlabel("Generation")
plt.ylabel("Average Fitness")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define a function to run the genetic algorithm with different numbers of generations
def run_experiment(num_generations):
    avg_fitness_per_generation, _ = genetic_algorithm(num_generations=num_generations)
    return avg_fitness_per_generation

# Define numbers of generations for the experiments
num_generations_list = [10, 100, 500, 1000]

# Initialize a dictionary to store average fitness per generation for each number of generations
avg_fitness_per_generation_count = {}

# Run experiments for each number of generations
for num_generations in num_generations_list:
    avg_fitness_per_generation_count[num_generations] = run_experiment(num_generations)

# Plotting
generations = np.arange(1, max(num_generations_list) + 1)

plt.figure(figsize=(10, 6))

# Plot average fitness per generation for each number of generations
for num_generations, avg_fitness in avg_fitness_per_generation_count.items():
    plt.plot(generations[:len(avg_fitness)], avg_fitness, label=f"Generations: {num_generations}")

plt.title("Average Fitness per Generation for Different Numbers of Generations")
plt.xlabel("Generation")
plt.ylabel("Average Fitness")
plt.legend()
plt.grid(True)
plt.show()
